In [1]:
import re
import moval
import itertools
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Let's check the model parameters first (pickle)

In [31]:
# pickle_path_org = 'HAM_classification_auc_max_class_probability-conf_ts-model_False.pkl'
# pickle_path_cs = 'HAM_classification_auc_max_class_probability-conf_ts-model_True.pkl'
# pickle_path_org = 'Cardiac_segmentation_f1score_max_class_probability-conf_atc-model_False.pkl'
# pickle_path_cs = 'Cardiac_segmentation_f1score_max_class_probability-conf_atc-model_True.pkl'
pickle_path_org = 'Cifar10ci2_classification_accuracy_energy-conf_ts-model_False.pkl'
pickle_path_cs = 'Cifar10ci2_classification_accuracy_energy-conf_ts-model_True.pkl'

In [32]:
with open(pickle_path_org, 'rb') as file:
    loaded_ckpt = pickle.load(file)
org_param = loaded_ckpt["param"]
print(org_param)

[9.08412657]


In [33]:
np.mean(loaded_ckpt["fitted_perf"])

0.740666665282634

In [34]:
len(loaded_ckpt["fitted_perf"])

3000

In [35]:
with open(pickle_path_cs, 'rb') as file:
    loaded_ckpt = pickle.load(file)
cs_param = loaded_ckpt["param"]
print(cs_param)

[ 7.10525036  7.96918507  9.00384159  6.98718138 10.17681732  9.31715851
 11.0485713  11.42319136 11.99084339 12.05871696]


In [36]:
print(np.mean(loaded_ckpt["fitted_perf"]))

0.7406666637319568
